# Function Calling

Function calling allows you to connect external tools to the model. This technique is specially useful when you used with CoT prompting or ReAct prompting for tasks which uses external tools.

## Steps to use function calling:

### 1. Define functions which will be used as tools

In [58]:
budget_limits = {
    "food": 500,
    "entertainment": 200,
    "electronics": 1000,
    "clothing": 300,
    "miscellaneous": 150
}

current_spending = {
    "food": 40,
    "entertainment": 10,
    "electronics": 80,
    "clothing": 0,
    "miscellaneous": 0
}

expense_keywords = {
    "food": ["grocery", "restaurant", "food", "meal", "snack"],
    "entertainment": ["movie", "concert", "game", "book", "music"],
    "electronics": ["computer", "phone", "gadget", "laptop", "tablet"],
    "clothing": ["shirt", "pants", "shoes", "jacket", "dress"],
    "miscellaneous": ["gift", "household", "stationery"]
}

def calculate(expression):
    """Performs basic arithmetic operations."""
    try:
        return eval(expression)
    except:
        return "Invalid expression"

def get_expense_category(description):
    """Categorizes an expense based on keywords."""
    description = description.lower()
    for category, keywords in expense_keywords.items():
        if any(keyword in description for keyword in keywords):
            return category
    return "miscellaneous"

def get_budget_limit(category):
    """Returns the budget limit for a given category."""
    return budget_limits.get(category, 0)

def get_current_spending(category):
    """Returns the current spending for a given category."""
    return current_spending.get(category, 0)

def add_expense(amount, description):
    """Adds an expense to the current spending."""
    category = get_expense_category(description)
    current_spending[category] += amount
    return f"Added ${amount} to {category} category."

def get_remaining_budget(category):
    """Calculates the remaining budget for a category."""
    limit = get_budget_limit(category)
    spent = get_current_spending(category)
    return limit - spent

def check_budget_status(category):
    """Checks the budget status for a category."""
    limit = get_budget_limit(category)
    spent = get_current_spending(category)
    remaining = limit - spent
    percentage = (spent / limit) * 100 if limit > 0 else 0
    return f"Category: {category}\nSpent: ${spent}\nBudget: ${limit}\nRemaining: ${remaining}\nPercentage used: {percentage:.2f}%"

tools_available = {
    "calculate": calculate,
    "get_expense_category": get_expense_category,
    "get_budget_limit": get_budget_limit,
    "get_current_spending": get_current_spending,
    "get_remaining_budget": get_remaining_budget,
    "add_expense": add_expense,
    "check_budget_status": check_budget_status
}


### 2. Describe the functions so the model know how to use it.

In [59]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "calculate",
            "description": "Performs basic arithmetic operations (add, subract, multiply, divide) on the provided mathematical expression.",
            "parameters": {
                "type": "object",
                "properties": {
                    "expression": {
                        "type": "string",
                        "description": "A string containing a mathematical expression, e.g., '2 + 3 * 4'."
                    }
                },
                "required": ["expression"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "get_expense_category",
            "parameters": {
                "type": "object",
                "properties": {
                    "description": {
                        "type": "string",
                        "description": "Categorizes an expense based on keywords found in the description. Matches against predefined categories like food, entertainment, electronics, clothing, and miscellaneous"
                    }
                },
                "required": ["description"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "get_budget_limit",
            "description": "Returns the budget limit for a given category based on predefined limits.",
            "parameters": {
                "type": "object",
                "properties": {
                    "category": {
                        "type": "string",
                        "description": "A string representing the category, e.g., 'food', 'electronics'."
                    }
                },
                "required": ["category"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "get_current_spending",
            "description": "Returns the current spending amount for a given category.",
            "parameters": {
                "type": "object",
                "properties": {
                    "category": {
                        "type": "string",
                        "description": "A string representing the category, e.g., 'clothing', 'entertainment'."
                    }
                },
                "required": ["category"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "add_expense",
            "description": "Adds a specified expense amount to the current spending of the appropriate category, determined based on the description.",
            "parameters": {
                "type": "object",
                "properties": {
                    "amount": {
                        "type": "integer",
                        "description": "A number representing the expense amount, e.g., 50."
                    },
                    "description": {
                        "type": "string",
                        "description": "A string describing the expense, e.g., 'Dinner at a restaurant'."
                    }
                },
                "required": ["amount", "description"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "get_remaining_budget",
            "description": "Calculates the remaining budget for a specified category by subtracting the current spending from the budget limit.",
            "parameters": {
                "type": "object",
                "properties": {
                    "category": {
                        "type": "string",
                        "description": "A string representing the category, e.g., 'miscellaneous', 'food'."
                    }
                },
                "required": ["category"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "check_budget_status",
            "description": "Checks the budget status for a specified category, providing details on the total budget, amount spent, remaining budget, and percentage of budget used.",
            "parameters": {
                "type": "object",
                "properties": {
                    "category": {
                        "type": "string",
                        "description": "A string representing the category, e.g., 'electronics', 'food'."
                    }
                },
                "required": ["category"]
            }
        }
    }
]

### 3. Generate Response

In [60]:
from dotenv import load_dotenv
from openai import OpenAI
import json

load_dotenv("../.env")
client = OpenAI()


class Agent:
    def __init__(self, tools, model="gpt-4o-mini", verbose: bool = True):
        self.tools = tools
        self.model = model
        self.verbose = verbose

    def complete(self, messages: list):
        response = client.chat.completions.create(
            messages=messages,
            model=self.model,
            tools=self.tools
        )
        return response.choices[0].message

    def query(self, question: str):
        messages = [
            {"role": "system", "content": "You are a helpful customer support assistant. Use the supplied tools to assist the user. You MUST use only one tool at a time"},
            {"role": "user", "content": question}
        ]

        while True:
            response = self.complete(messages)
            messages.append(response)

            if response.tool_calls:
                for tool_call in response.tool_calls:
                    func_name, func_args = tool_call.function.name, json.loads(tool_call.function.arguments)
                    print(f"Action: {func_name}({func_args})") if self.verbose else None
                    tool_response = tools_available[func_name](**func_args) 
                    print(f"Observation: {tool_response}\n{'-'*10}") if self.verbose else None
                    messages.append(
                        {
                            "tool_call_id": tool_call.id,
                            "role": "tool",
                            "name": func_name,
                            "content": str(tool_response),
                        }
                    )
            else:
                return response.content

In [62]:
questions = [
    "I just spent $40 on dinner at a restaurant. How much of my food budget is left for the month?",
    "If I buy a $900 laptop and a $200 smartphone, will I exceed my electronics budget?",
    "What percentage of my entertainment budget have I used so far, and how much is left?",
    "I'm planning to buy clothes for $250 and a gift for $100. Can I afford both within my respective budgets?",
    "I've just spent $150 on groceries and $50 on a restaurant meal this week. How much more can I spend on food this month?",
    "What categories have I overspent on, and by how much?"
]
question = questions[1]
print(f"Question: {question}")

agent = Agent(tools)
response = agent.query(question)

print(response)

Question: If I buy a $900 laptop and a $200 smartphone, will I exceed my electronics budget?
Action: get_budget_limit({'category': 'electronics'})
Observation: 1000
----------
Action: get_current_spending({'category': 'electronics'})
Observation: 80
----------
Your electronics budget limit is $1000, and your current spending is $80. 

If you buy a $900 laptop and a $200 smartphone, your total spending will be $80 + $900 + $200 = $1180.

Since $1180 exceeds your budget limit of $1000, you will indeed exceed your electronics budget.
